Notebook for preliminary analysis, planned function includes:

1.Finding sensor readouts that exceeds 850.(Done)

2.Finding abnormal sensor readouts in one group.(Done. Now using 20% deviation from average.)

3.Finding relationships between prediction results, threshold and command.(TBD)

4.Refactoring the dataset. e.g. Combine sensor values and mv values into the same row.

In [ ]:
import pandas as pd
cleaned_file_path = ''
df = pd.read_csv(cleaned_file_path)
print("Loaded DataFrame:")
print(df.head())

In [ ]:
sensor_2_saturated = df[df['sensor_2'] > 850]
print("\nRows where sensor_2 > 850:")
print(sensor_2_saturated)

In [ ]:
sensor_4_saturated = df[df['sensor_4'] > 850]
print("\nRows where sensor_4 > 850:")
print(sensor_4_saturated)

In [ ]:
threshold = 0.2  
abnormal_mv_rows = []

for index, row in df.iterrows():
    mv_values = [row['mv_1'], row['mv_2'], row['mv_3'], row['mv_4']]
    avg_mv = sum(mv_values) / len(mv_values)

    for mv in mv_values:
        if abs(mv - avg_mv) / avg_mv > threshold:
            abnormal_mv_rows.append(row)
            break

abnormal_mv_df = pd.DataFrame(abnormal_mv_rows)
print("\nRows with abnormal mv readings:")
print(abnormal_mv_df)